In [147]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms

In [4]:
# 기본구조
# 1. 파일 경로와 정답라벨(0~17) 각각 Series로 생성
# 1-1. 라벨링 함수 제작
# 2. 옵션적용 (커스텀가능한 구조로)

In [41]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

In [121]:
# 일단은 list로 값 받는 형태로 _ Dataset 안에서 처리할 수도 있을듯
whole_image_path = []
whole_target_label = []

for path in dt_train['path']:
    for file_name in [i for i in os.listdir(train_image_path+path) if '._' not in i]:
        whole_image_path.append(train_image_path+path+'/'+file_name)
        whole_target_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))
        
        
# 라벨을 0~17로 할당하는 함수
def onehot_enc(x):
    # x 입력형태 : (gender, age, mask)의 튜플
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

(512, 384, 3)

In [122]:
# sr_data : 이미지 데이터의 경로
# sr_label : 이미지 데이터의 라벨정보 (not_encoded)
sr_data = pd.Series(whole_image_path)
sr_label = pd.Series(whole_target_label)
#64, 447

In [175]:
class Dataset_Mask(Dataset):
    def __init__(self, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = sr_data
        self.label = sr_label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(sr_data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[64:447]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [176]:
dataset_mask = Dataset_Mask(transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [177]:
DataLoader_Mask = DataLoader(dataset = dataset_mask, batch_size=16, shuffle=True, num_workers=0)